## Hello Cassandra

### Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating the database
(artist_name, song_title, first_name,last_name, \
  user_id, session_id, item_in_session) = line[1], line[4], line[3]

#### Creating a Cluster

In [70]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [71]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify 
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [72]:
session.set_keyspace("sparkify")

#### Now we create tables to run the following queries, because, with Apache Cassandra you model the database tables on the queries to be ran.

#### Create queries to ask the following three questions of the data

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Table 1

Here, the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. So the partition is done by sessionId and within that partition, rows are ordered by the itemInSession.

In [43]:
#Table for query 1
query = "CREATE TABLE IF NOT EXISTS session_library"
query = query + "(artist_name text, song_title text, song_length double, session_id int, item_in_session int, \
                 PRIMARY KEY (session_id, item_in_session))"

session.execute(query);            

In [33]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #INSERT
        query = "INSERT INTO session_library (artist_name, song_title, song_length, session_id, item_in_session)"
        query = query + "VALUES (%s, %s, %s , %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

**REFERENCE**: Explicit type conversions when using csv reader
https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python

#### Using a SELECT to verify that the data have been inserted into each table

In [62]:
query = "SELECT artist_name, song_title, song_length FROM session_library WHERE session_id = 338 AND item_in_session = 4"
rows = session.execute(query)

for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

##### Table 2

Here, the Primary Key is combination of three fields: user_id and session_id are the composite partition key, and itemInSession is clustering key. So the partition is done by sessionId and within that partition, rows are ordered by the itemInSession.

In [77]:
query = "CREATE TABLE IF NOT EXISTS user_session_library"
query = query + "(artist_name text, song_title text, first_name text, last_name text, user_id int, session_id int, \
                 item_in_session int, PRIMARY KEY ((user_id, session_id), item_in_session))"
session.execute(query) 

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_library (artist_name, song_title, first_name, last_name, \
                user_id, session_id, item_in_session)"
        query = query + "VALUES (%s, %s, %s , %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))


**Validating Table 2**

In [78]:
query = "SELECT artist_name, song_title, first_name, last_name FROM user_session_library\
         WHERE user_id = 10 AND session_id = 182"
rows = session.execute(query)

for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz



#### Table 3

Here, the Primary Key has two fields: song_title is the partition key, and user_id is clustering key. So the partition is done by song_title and within that partition, rows are ordered by the itemInSession.

In [1]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 
#'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS listen_history"
query = query + "(first_name text, last_name text, song_title text, user_id int, \
                 PRIMARY KEY (song_title, user_id))"
session.execute(query)     

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #INSERT
        query = "INSERT INTO listen_history (first_name, last_name, song_title, artist_name, user_id)"
        query = query + "VALUES (%s, %s, %s , %s, %s)"
        session.execute(query, (line[1], line[4], line[9], line[0], int(line[10])))



#### validating query with select statements

In [98]:
query = "SELECT first_name, last_name FROM listen_history \
         WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query)

for row in rows:
    print(row.first_name, row.last_name, row.song_title, row.artist_name)

Jacqueline Lynch All Hands Against His Own The Black Keys
Tegan Levine All Hands Against His Own The Black Keys
Sara Johnson All Hands Against His Own The Black Keys


### Validating tables with Pandas

In [99]:
event_table = pd.read_csv("event_datafile_new.csv")
event_table.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [100]:
#Query 1 is correct
event_table[(event_table.sessionId == 338) & (event_table.itemInSession == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [58]:
#Query 2
event_table[(event_table.userId == 10) & (event_table.sessionId == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [101]:
#query 3
event_table[event_table.song == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [102]:
query = "DROP TABLE IF EXISTS session_library"
session.execute(query);

query = "DROP TABLE IF EXISTS user_session_library"
session.execute(query);

query = "DROP TABLE IF EXISTS listen_history"
session.execute(query);

### Close the session and cluster connection¶

In [103]:
session.shutdown()
cluster.shutdown()